# IRP Integration Demo

This notebook showcases usage of the `irp_integration` module. 

### Imports & Setup

In [7]:
%load_ext autoreload
%autoreload 2
from helpers.irp_integration import IRPClient
import json, os
from pathlib import Path
from helpers.irp_integration.utils import extract_analysis_id_from_workflow_response, get_nested_field

irp_client = IRPClient()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Configurations

In [2]:
edm_name = "CBHU_Automated" # ensure unique within IRP
portfolio_name = "CBHU_Automated" # ensure unique within the edm
accounts_file_name = "Modeling_202503_Moodys_CBHU_Account.csv"
locations_file_name = "Modeling_202503_Moodys_CBHU_Location.csv"
analysis_job_name = "CBHU_Analysis"
analysis_profile_name = "DLM CBHU v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
event_rate_scheme_name = "RMS 2025 Stochastic Event Rates"

### Create EDM

In [6]:
print(json.dumps(irp_client.edm.create_edm(edm_name=edm_name, server_name="databridge-1"), indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23009514
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23009514
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23009514
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23009514
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23009514
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23009514
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23009514
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/risk

### Create Portfolio

In [ ]:
portfolio_response = irp_client.portfolio.create_portfolio(edm_name=edm_name, portfolio_name=portfolio_name, portfolio_number=portfolio_name)
portfolio_id = get_nested_field(portfolio_response, 'id', required=True, context="get portfolio id")
print(json.dumps(portfolio_response, indent=2))

{
  "id": 3
}


### MRI Import

In [6]:
# New simplified approach using import_from_files() convenience method
notebook_dir = Path(os.getcwd())
working_files_dir = notebook_dir / "workspace" / "workflows" / "_Tools" / "files" / "working_files"

import_response = irp_client.mri_import.import_from_files(
    edm_name=edm_name,
    portfolio_id=portfolio_id,
    accounts_file=accounts_file_name,
    locations_file=locations_file_name,
    mapping_file="mapping.json",
    working_dir=str(working_files_dir)
)

print(json.dumps(import_response, indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010579
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010579
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010579
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010579
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010579
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010579
Workflow status: RUNNING; Percent complete: 50
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010579
Workflow status: RUNNING; Percent complete: 50
Polling workflow url https://api-euw1.rms-ppe.com/riskmode

### Create Treaty

In [ ]:
# New simplified approach using create_treaty_from_names() convenience method
treaty_name = "CBHU_Treaty"

treaty_response = irp_client.treaty.create_treaty_from_names(
    edm_name=edm_name,
    treaty_name=treaty_name,
    treaty_type_name="Working Excess",
    currency_name="US Dollar",
    attachment_basis_name="Losses Occurring",
    attachment_level_name="Location",
    risk_limit=3000000,
    occur_limit=9000000,
    attach_pt=2000000,
    pcnt_covered=100,
    pcnt_placed=95,
    pcnt_ri_share=100,
    pcnt_retent=100,
    premium=0,
    num_of_reinst=99,
    reinst_charge=0,
    aggregate_limit=0,
    aggregate_deductible=0,
    priority=1,
    effect_date="2025-10-15T17:49:10.637Z",
    expire_date="2026-10-15T17:49:10.637Z",
    auto_assign_lobs=True
)
treaty_id = get_nested_field(treaty_response, 'id', required=True, context="get treaty id")

print(f"Treaty created successfully!")
print(f"Treaty ID: {treaty_id}")
print(f"LOBs assigned: {treaty_response.get('lobs_assigned', 'N/A')}")

Treaty created successfully!
Treaty ID: 1
LOBs assigned: 1


### Upgrade EDM Version

In [8]:
print(json.dumps(irp_client.edm.upgrade_edm_version(edm_name), indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010703
Workflow status: PENDING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010703
Workflow status: RUNNING; Percent complete: 70
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010703
Workflow status: FINISHED; Percent complete: 100
{
  "id": 23010703,
  "userName": "Partner - PremiumIQ key",
  "status": "FINISHED",
  "submitTime": "2025-10-29T17:10:36.220Z",
  "startTime": "2025-10-29T17:10:37Z",
  "endTime": "2025-10-29T17:10:48Z",
  "name": "CBHU_Automated",
  "type": "EDM_DATA_UPGRADE",
  "progress": 100,
  "priority": "medium",
  "entitlement": "RI-RISKMODELER",
  "resourceGroupId": "ba672cd9-3384-4cb3-9882-bbeb5f8478c6",
  "jobs": [
    {
      "id": "a1f0e9dc-11df-429d-ba1f-d79a2225c81d",
      "taskId": 1,
      "workflowId": 23010703,
      "status": "Succeeded",
      "submitTime": "2025

### GeoHaz

In [9]:
print(json.dumps(
    irp_client.portfolio.geohaz_portfolio(edm_name, 
                                          portfolio_id, 
                                          geocode=True, 
                                          hazard_eq=True, 
                                          hazard_ws=True), 
    indent=2
))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010799
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010799
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010799
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010799
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010799
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010799
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23010799
Workflow status: RUNNING; Percent complete: 55
Polling workflow url https://api-euw1.rms-ppe.com/risk

### Execute Single Analysis

In [ ]:
analysis_response = irp_client.analysis.execute_analysis(
    analysis_job_name,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name,
    [treaty_id]
)
print(json.dumps(analysis_response, indent=2))

### Execute Multiple Analyses

In [ ]:
analysis_profile_name = "DLM CBHU v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
tag = "cbhu_analyses"

analysis_job_name1 = "CBHU_Analysis 2023 Rates"
event_rate_scheme_name1 = "RMS 2023 Stochastic Event Rates"
workflow_id1 = irp_client.analysis.submit_analysis_job(
    analysis_job_name,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name1,
    [treaty_id],
    [tag]
)

analysis_job_name2 = "CBHU_Analysis 2025 Rates"
event_rate_scheme_name2 = "RMS 2025 Stochastic Event Rates"
workflow_id2 = irp_client.analysis.submit_analysis_job(
    analysis_job_name,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name2,
    [treaty_id],
    [tag]
)

workflow_ids = [workflow_id1, workflow_id2]
response = irp_client.analysis.poll_analysis_job_batch(workflow_ids)
print(json.dumps(response, indent=2))
analysis_ids = []
for workflow in response['workflows']:
    analysis_ids.append(extract_analysis_id_from_workflow_response(workflow))

{
  "currency": {
    "asOfDate": "2018-11-15",
    "code": "USD",
    "scheme": "RMS",
    "vintage": "RL18.1"
  },
  "edm": "CBHU_Automated",
  "eventRateSchemeId": 578,
  "exposureType": "PORTFOLIO",
  "id": 2,
  "modelProfileId": 4418,
  "outputProfileId": 123,
  "treaties": [
    1
  ],
  "tagIds": [
    1205
  ],
  "globalAnalysisSettings": {
    "franchiseDeductible": false,
    "minLossThreshold": "1.00",
    "treatConstructionOccupancyAsUnknown": true,
    "numMaxLossEvent": 1
  },
  "jobName": "CBHU_Analysis"
}
{
  "currency": {
    "asOfDate": "2018-11-15",
    "code": "USD",
    "scheme": "RMS",
    "vintage": "RL18.1"
  },
  "edm": "CBHU_Automated",
  "eventRateSchemeId": 739,
  "exposureType": "PORTFOLIO",
  "id": 2,
  "modelProfileId": 4418,
  "outputProfileId": 123,
  "treaties": [
    1
  ],
  "tagIds": [
    1205
  ],
  "globalAnalysisSettings": {
    "franchiseDeductible": false,
    "minLossThreshold": "1.00",
    "treatConstructionOccupancyAsUnknown": true,
    "nu

### Group Analyses

In [8]:
group_name = "CBHU Analysis Group"
print(json.dumps(irp_client.analysis.create_analysis_group(analysis_ids, group_name), indent=2))

{
  "analysisIds": [
    "33743",
    "33742"
  ],
  "name": "CBHU Analysis Group",
  "currency": {
    "asOfDate": "2018-11-15",
    "code": "USD",
    "scheme": "RMS",
    "vintage": "RL18.1"
  },
  "simulateToPLT": true,
  "numOfSimulations": 50000,
  "propagateDetailedLosses": false,
  "reportingWindowStart": "01/01/2021",
  "simulationWindowStart": "01/01/2021",
  "simulationWindowEnd": "12/31/2021",
  "regionPerilSimulationSet": [],
  "description": ""
}
Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23013042
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23013042
Workflow status: PENDING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23013042
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/23013042
Workflow status: RUNNING; Perce

### Export Analyses to RDM

In [ ]:
rdm_name = "CBHU_Automated_RDM"
export_response = irp_client.rdm.export_analyses_to_rdm(rdm_name, analysis_ids)
print(json.dumps(export_response, indent=2))